In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 8 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (9,700 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 120509 files and direc

In [ ]:
import pandas as pd
import numpy as np
import warnings
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
pd.set_option('display.float_format', lambda x: '%.2f' %x)
warnings.filterwarnings('ignore')

In [ ]:
import matplotlib.font_manager as fm
from matplotlib import rc

font_list = fm.findSystemFonts(fontpaths=None, fontext = 'ttf')
font_list

['/usr/share/fonts/truetype/liberation/LiberationSans-BoldItalic.ttf',
 '/usr/share/fonts/truetype/nanum/NanumMyeongjoBold.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Bold.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSerif-Regular.ttf',
 '/usr/share/fonts/truetype/nanum/NanumSquareRoundR.ttf',
 '/usr/share/fonts/truetype/nanum/NanumSquareRoundB.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf',
 '/usr/share/fonts/truetype/nanum/NanumBarunGothicBold.ttf',
 '/usr/share/fonts/truetype/humor-sans/Humor-Sans.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothic.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothicCodingBold.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationMono-Bold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumSquareR.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationMono-Italic.ttf',
 '

In [ ]:
font_name = fm.FontProperties(fname='/usr/share/fonts/truetype/nanum/NanumBarunGothicBold.ttf').get_name() #font_list에 있는 경로 중 하나로 설정하기
plt.rc('font', family = font_name)

In [ ]:
liar_data = pd.read_csv('/DATA/DATA01_ENC.csv',index_col=0)
data02 = pd.read_csv('/DATA/DATA02_ENC.csv',index_col=0)
data04 = pd.read_csv('/DATA/DATA04_ENC.csv',index_col=0)
data01_norm = pd.read_csv('/DATA/DATA01_NORM.csv',index_col=0)
data02_norm = pd.read_csv('/DATA/DATA02_NORM.csv',index_col=0)
data04_norm = pd.read_csv('/DATA/DATA04_NORM.csv',index_col=0)

In [ ]:
data01 = pd.concat([liar_data, data01_norm])
liar = list(liar_data['CUS_ACN_ENC'])
data= pd.concat([data02, data04, data02_norm, data04_norm])


In [ ]:
data['CNAC_DPSR_NM'] = data['CNAC_DPSR_NM'].str.replac("-",'')
data['CNAC_DPSR_NM'] = data['CNAC_DPSR_NM'].str.replac("_",'')
data['CNAC_DPSR_NM'] = data['CNAC_DPSR_NM'].str.strip()

In [ ]:
data_cash = data[data['NODES']=='']
data_normal = data[data['NODES']!='']
data_cash['NODES'] = 'CASH' + data['CASE_ID'].astype('str')
data = pd.concat([data_cash,data_normal])

In [ ]:
cash = set(data_cash['NODES'])

In [ ]:
def df_pp(df):
  df_net = df[['CUS_ACN_ENC','NODES', 'LOG_AMT','DWM_DCD','CASE_ID', 'MIN_TRN_YMD', 'MAX_TRN_YMD', 'RMTN_RQTR_NM', 'CNAC_DPSR_NM']]

  df_net.rename(columns = {'CUS_ACN_ENC':'to','NODES':'from'},inplace=True)

  mask = df_net['DWM_DCD'] == 2
  df_net.loc[mask, ['from','to']] = df_net.loc[mask,['to','from']].values
  df_net.drop(['DWM_DCD'],inplace=True,axis=1)

  from sklearn.preprocessing import MinMaxScaler

  scaler = MinMaxScaler()
  df_net['LOG_AMT_minmax'] = scaler.fit_transform(df_net[['LOG_AMT']]) + 0.0001
  df_net['MIN_TRN_YMD'] = pd.to_datetime(df_net['MIN_TRN_YMD'])
  df_net['MAX_TRN_YMD'] = pd.to_datetime(df_net['MAX_TRN_YMD'])

  df_net['edge'] = list(zip(df_net['from'],df_net['to']))
  df_net['weighted_edge'] = list(zip(df_net['from'], df_net['to'], df_net['LOG_AMT_minmax']))

  return df_net

In [ ]:
rmtn_cnt = data.groupby(['CASE_ID','RMTN_RQTR_NM'])['RMTN_RQTR_NM'].count()
rmtn_cnt = rmtn_cnt.reset_index(name = 'Count')
rmtn_cnt = rmtn_cnt[rmtn_cnt['RMTN_RQRT_NM']!='']
rmtn_cnt = rmtn_cnt[rmtn_cnt['Count']>1]

grouped = rmtn_cnt.groupby('CASE_ID')
rmtn_dict = {}

def abb_data(df,case_id):
  df = df[df['CASE_ID']==case_id]
  all_nodes = set(df['to']).union(set(df['from']))
  case_liar = set(liar_data['CUS_ACN_ENC'])&set(all_nodes)
  not_del = ['대부', '크레디트', '파이낸스','파이낸셜', '캐피탈', '코인', '주식','유한','비트']
  df1 = df[df['RMTN_RQTR_NM'].str.contains('|'.join(not_del))|df['RMTN_RQTR_NM'].isin(rmtn_dict[case_id])|df['RMTN_RQTR_NM'].str.match('^[a-zA-Z\s]+$')]
  df2 = df[~(df['RMTN_RQTR_NM'].str.contains('|'.join(not_del))|df['RMTN_RQTR_NM'].isin(rmtn_dict[case_id])|df['RMTN_RQTR_NM'].str.match('^[a-zA-Z\s]+$'))]
  df2 = df2[df2['to'].isin(case_liar)|df2['from'].isin(case_liar)|df2['to'].str.contains('CASH')|df2['from'].str.contains('CASH')]
  df = pd.concat([df1,df2])
  return df

In [ ]:
import math
def make_coor(angle):
  x = math.cos(math.pi*angle)
  y = math.sin(math.pi*angle)
  return [x,y]
def multi_radius(coor,r):
  return [i*r for i in coor]
def rotate(x,angle): #rotation matrix 곱하기
  x1, x2 = x[0],x[1]
  return [math.cos(math.pi*(angle-1/2))*x1-math.sin(math.pi*(angle-1/2))*x2, math.sin(math.pi*(angle-1/2))*x1 + math.cos(math.pi*(angle-1/2))*x2]

In [ ]:
def pos1(dnet,case_id):
  seed = list(data01[data01['CASE_ID']==case_id]['CUS_ACN_ENC'])[0]

  # 각도 정해주기
  all_nodes = list((set(pr['to'])).union(set(pr['from'])))
  all_nodes.remove(seed)
  dic_min = {}
  for node in all_nodes:
    min_date = pr[(pr['to']==node)|(pr['from']==node)]['MIN_TRN_YMD'].min()
    dic_min[node] = min_date

  dic_df = pd.DateFrame(columns = ['NODE','DATE'])
  for key, value in dic_min.items():
    new_data = {'NODE':key, 'DATE':value}
    dic_df = dic_df.append(new_data, ignore_index = True)

  dic_df = dic_df.sort_values(by='DATE',ascending = False)
  dic_df.reset_index(inplace=True)

  dic_df['order'] = (dic_df.index + 1)/(len(dic_df)+1)
  dic_df['coor'] = dic_df['order'].apply(make_coor)

  depth1_nodes = set(pr[(pr['to']==seed)|(pr['from']==seed)]['to']).union(set(pr[(pr['to']==seed)|(pr['from']==seed)]['from']))
  depth2_nodes = set(pr[(pr['to']!=seed)&(pr['from']!=seed)]['to']).union(set(pr[(pr['to']!=seed)&(pr['from']!=seed)]['from']))
  depth1and2 = depth2_nodes & depth1_nodes #depth1과 depth 둘다에 속하는 노드들
  depth2_nodes = depth2_nodes - depth1_nodes #depth1에 속하지 않고 depth2에만 속하는 노드들

  dic_df.loc[dic_df['NODE'].isin(depth1and2),'coor'] = dic_df.loc[dic_df['NODE'].isin(depth1and2),'coor'].apply(multi_radius,args =[2]) #반지름 길이 2
  dic_df.loc[dic_df['NODE'].isin(depth2_nodes),'coor'] = dic_df.loc[dic_df['NODE'].isin(depth2_nodes),'coor'].apply(multi_radius,args =[3]) #반지름 길이 3

  pos = {seed: [0,0]} #노드 : 좌표
  for i in range(len(dic_df)):
    pos[dic_df.loc[i]['NODE']] = dic_df.loc[i]['coor']

  return pos

In [ ]:
def pos2(dnet,case_id):
  seed = list(data01[data01['CASE_ID']==case_id]['CUS_ACN_ENC'])[0]

  # 각도 정해주기
  all_nodes = list((set(pr['to'])).union(set(pr['from'])))
  all_nodes.remove(seed)
  dic_min = {}
  for node in all_nodes:
    min_date = pr[(pr['to']==node)|(pr['from']==node)]['MIN_TRN_YMD'].min()
    dic_min[node] = min_date

  dic_df = pd.DateFrame(columns = ['NODE','DATE'])
  for key, value in dic_min.items():
    new_data = {'NODE':key, 'DATE':value}
    dic_df = dic_df.append(new_data, ignore_index = True)

  dic_df = dic_df.sort_values(by='DATE',ascending = False)
  dic_df.reset_index(inplace=True)

  dic_df['order'] = (dic_df.index + 1)/(len(dic_df)+1)
  dic_df['coor'] = dic_df['order'].apply(make_coor)

  depth1_nodes = set(pr[(pr['to']==seed)|(pr['from']==seed)]['to']).union(set(pr[(pr['to']==seed)|(pr['from']==seed)]['from']))
  depth2_nodes = set(pr[(pr['to']!=seed)&(pr['from']!=seed)]['to']).union(set(pr[(pr['to']!=seed)&(pr['from']!=seed)]['from']))
  depth1and2 = depth2_nodes & depth1_nodes #depth1과 depth 둘다에 속하는 노드들
  depth2_nodes = depth2_nodes - depth1_nodes #depth1에 속하지 않고 depth2에만 속하는 노드들

  dic_df.loc[dic_df['NODE'].isin(depth1and2),'coor'] = dic_df.loc[dic_df['NODE'].isin(depth1and2),'coor'].apply(multi_radius, args[2.2])

  for node in depth2_nodes:
    if len(pr[pr['to']==node]) != 0:
      match_node = list(pr[pr['to']==node])[0]
    elif len(pr[pr['from']==node]) != 0:
      match_node = list(pr[pr['from']==node])[0]
    add_coor = dic_df.loc[dic_df['NODE']==match_node, 'coor'].values[0]
    angle = dic_df.loc[dic_df['NODE'] == match_node]['order'].values[0]
    dic_df.loc[dic_df['NODE']==node, 'coor'] = dic_df.loc[dic_df['NODE']==node, 'coor'].apply(add_and_rotate, args = [angle])
    dic_df.loc[dic_df['NODE']==node, 'coor'] = dic_df.loc[dic_df['NODE']==node, 'coor'].apply(lambda x: [x[0]*0.5 + add_coor[0], x[1]*0.5+add_coor[1]])

    pos = {seed : [0,0]}
    for i in range(len(dic_df)):
      pos[dic_df.loc[i]['NODE']] = dic_df.loc[i]['coor']

    return pos

In [ ]:
def color_apply(i,color_range):
  viridis_cmap = plt.get_cmap('plasma') #viridis 혹은 hsv 등 다른 그라데이션으로 대체 가능
  values = [x/color_range for x in range(color_range+1)]
  colors = viridis_cmap(values)
  hex_colors =[mcolors.rgb2hex(color) for color in colors]
  return hex_color[i]

def case_nodes_edges(df,case_id):
  case_net = df[df['CASE_ID']==case_id]
  case_net = abb_data(df,case_id)

  case_net = case_net.sort_values(by = 'MIN_TRN_YMD')
  case_net.reset_index(inplace=True)

  min_min = case_net['MIN_TRN_YMD'].min()
  case_net['Edge_colors'] = case_net['MIN_TRN_YMD']-min_min
  case_net['Edge_colors'] = case_net['Edge_colors'].astype(str)
  case_net['Edge_colors'] = case_net['Edge_colors'].str.extract('(\d+)',expand = False)
  case_net['Edge_colors'] = case_net['Edge_colors'].astype(int)

  color_range = case_net['Edge_colors'].max()
  case_net['Edge_colors'] = case_net['Edge_colors'].apply(color_apply,args=[color_range])

  nodes = set(case_net['to'].append(case_net['from']))
  edges = list(case_net['weighted_edge'])
  edge_colors = list(case_net['Edge_colors'])

  if pos == 1:
    position = pos1(case_net, case_id)
  elif pos == 2:
    position = pos2(case_net, case_id)

  return case_net, nodes, edges, position

In [ ]:
def rmtn_match(data,case_id,nodes):
  df_case = data[data['CASE_ID']==case_id]
  node_rmtn = dict()
  for i in range(len(nodes)):
    node_acn = list(nodes[i])
    if 'CASH' in node_acn:
      rmtn = node_acn
    else:
      a = df_case[df_case['NODES'].isin(set(nodes))]['NODES'] == node_acn
      if len(a[a])==0: #CUS에 들어가 있는 경우 or 다른 사기계좌의 NODES일 경우
        b = data[data['NODES']==node_acn]
        if b['RMTN_RQTR_NM'].nunique()==1:
          rmtn = b['RMTN_RQTR_NM'].values[0]
        else: #적요명 2개 이상인 경우, 계좌주명에서 찾기
          if b['CNAC_DPSR_NM'].nunique() == 0:
            rmtn = node_acn
          elif b['CNAC_DPSR_NM'].nunique() == 1:
            rmtn = b['CNAC_DPSR_NM'].values[0]
          elif b['CNAC_DPSR_NM'] == 2:
            cnac_lst = b['CNAC_DPSR_NM'].value_counts().index
            rmtn = [cnac for cnac in cnac_lst if cnac != ''][0]
          else:
            rmtn = 'rmtn 2over'
      else: #NODES 에 있는 경우
      c = df_case[df_case['NODES']==node_acn]
      if c['RMTN_RQTR_NM'].nunique() == 1:
        rmtn = c['RMTN_RQTR_NM'].values[0]
      else:
        if c['CNAC_DPSR_NM'].nunique() ==0:
          rmtn = node_acn
        elif c['CNAC_DPSR_NM'].nunique() ==1:
          rmtn = c['CNAC_DPSR_NM'].values[0]
        elif c['CNAC_DPSR_NM'].nunique() ==2:
          cnac_lst = b['CNAC_DPSR_NM'].value_counts().index
          rmtn = [cnac for cnac in cnac_lst if cnac != ''][0]
        else:
          rmtn = 'rmtn 2over'
    node_rmtn[node_acn] = rmtn
  return node_rmtn

In [ ]:
def draw_graph(df,case_id, pos = 2, layout = False, weight = False, label=True):
  case_net, nodes, edges, position = case_nodes_edges(df,case_id)

  G1 = nx.DiGraph()
  G1.add_nodes_from(nodes)
  G1.add_weighted_edges_from(edges)

  #노드 컬러 지정하기
  seed = list(data01[data01['CASE_ID']==case_id]['CUS_ACN_ENC'][0])
  node_colors = []
  for node in G1.nodes:
    if node == seed:
      if seed in list(liar_data['CUS_ACN_ENC']):
        node_colors.append('red') #사기
      else:
        node_colors.append('limegreen') #정상
    elif 'CASH' in node:
      node_colors.append('dodgerblue') #현금
    elif node not in list(liar_data['CUS_ANC_ENC']):
      node_colors.append('lightblue')
    else:
      node_colors.append('#fc8d62') #사기

  #엣지 컬러 지정하기
  edge_colors = [list(case_net[case_net['edge']==edges]['Edge_colors'])[0] for edges in G1.edges]
  if layout == True:
    pos = nx.fruchterman_reingold_layout(G1,iterations = 20) # 다른 layout 지정 가능
  else:
    pos = position

  nx.draw_networkx_nodes(G1, pos, node_size = 200, node_color = node_colors)

  if weight == True:
    edge_widths = [d['weight']*10 for u,v,d in G1.edges(data=True)]
    nx.draw_networkx_edges(G1, pos, arrowsize = 20, width = edge_widths, edge_color = edge_colors)
  else:
    nx.draw_networkx_edges(G1, pos, arrowsize = 20, edge_color = edge_colors)

  if label = True:
    node_rmtn = rmtn_match(data,case_id, G1.nodes)
    nx.draw_networkx_labels(G1,pos, font_family = font_name, font_size = 10, labels = node_rmtn)

  plt.axis('off')
  plt.title(f'CASE_ID : {case_id}')
  plt.show()

In [ ]:
d_net = df_pp(data)
draw_graph(d_net,222,label=False)

In [ ]:
draw_graph(d_net,222, weight = True, label=False)